# **🏖️ SANDCASTL 🏖️**

### **✨ Input Requirements ✨**
### ----------------------------

In [ ]:
# Full Directory Path for Input Spectrum (WAVELENGTH, FLUX, NOISE)
input_file = 

# Input outfile file name (DO NOT PUT DIRECTORY OR FILETYPE)
output_file = 

# Input the estimated spectral type (M0 = 0, L0 = 10, T0 = 20, NO GREATER THAN 30)
spt_estimate = 

# Set the number of threads used
thread = 

# Set the number of walkers
walkers = 

# Set the number of steps
steps = 

# Set the number of discards
discard = 

### ----------------------------

#### Imports All Required Packages

In [ ]:
# Imports all required packages 
import io
import os
import csv
import splat
import emcee
import corner
import contextlib
import numpy as np
import pandas as pd
from PIL import Image
import splat.model as spmdl
from astropy import units as u
import matplotlib.pyplot as plt
from specutils import Spectrum1D
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

#### Interpolation Between Grid Points

In [ ]:
# Interpolates between grid points so that the MCMC simulation can fit a more accurate model
def interpolation(mcmc_parm, real_parm, observed_wave, wavemove):
    
    # Gets the lower and upper bounds in the grid space
    lower_temp, upper_temp = real_parm[0], real_parm[1]
    lower_logg, upper_logg = real_parm[2], real_parm[3]
    lower_metal, upper_metal = real_parm[4], real_parm[5]

    # Gets the desired location to interpolate to
    x, y, z = mcmc_parm[0], mcmc_parm[1], mcmc_parm[2]

    # Makes a list of all of the required spectra on the grid
    model_list = [(spmdl.getModel(teff = lower_temp, logg = lower_logg, z = lower_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = upper_temp, logg = lower_logg, z = lower_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = lower_temp, logg = upper_logg, z = lower_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = upper_temp, logg = upper_logg, z = lower_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = lower_temp, logg = lower_logg, z = upper_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = upper_temp, logg = lower_logg, z = upper_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = lower_temp, logg = upper_logg, z = upper_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz')),
                  (spmdl.getModel(teff = upper_temp, logg = upper_logg, z = upper_metal, co = 0.55, cld = 'o0.55', logkzz = 2, set = 'lowz'))]

    # Resamples the wavelength regions of the model to be the same as the observed spectrum
    for i in range(len(model_list)): 
        model_wave = model_list[i].wave
        model_flux = model_list[i].flux
        input_spec = Spectrum1D(spectral_axis=model_wave, flux=model_flux) 
        f = interp1d(model_wave, model_flux, kind='linear')
        resampled_fluxes = f(observed_wave)
        model_list[i] = np.log10(resampled_fluxes)
        
    # Performs the interpolation
    f_intern = interpolate.griddata(([lower_temp, upper_temp, lower_temp, upper_temp, lower_temp, upper_temp, lower_temp, upper_temp], 
                                   [lower_logg, lower_logg, upper_logg, upper_logg, lower_logg, lower_logg, upper_logg, upper_logg], 
                                   [lower_metal, lower_metal, lower_metal, lower_metal, upper_metal, upper_metal, upper_metal, upper_metal]), 
                                   model_list, (x, y, z), method = 'linear', rescale = True)
    
    # Returns the interpolated fluxes in correct units
    f_intern = [x**10 for x in f_intern]
    return f_intern

#### $\chi^2$ Statistical Test Function
#### Formula: $\chi^2 = \frac{1}{N-P}\sum_{i = 1}^{N} \frac{(O_{i} - E_{i})^2}{\sigma_{i}^2}$

In [ ]:
# Rounds to the nearest hundred
def closest_hundreds(value):
    lower_hundred = (value // 100) * 100
    upper_hundred = lower_hundred + 100
    return lower_hundred, upper_hundred

# Rounds to the nearest half
def closest_bounds(target, numbers):
    lower_bound = max(num for num in numbers if num <= target)
    upper_bound = min(num for num in numbers if num >= target)
    return lower_bound, upper_bound

# Rounds to either -0.5 or -1.5
def round_to_negative_half_or_one_half(number):
    rounded_value = round(number) - 0.5 if number - int(number) < 0.5 else round(number) - 1.5
    return rounded_value

# Defines the chi^2 statistical approach for our MCMC simulation
def chi_square(observed_wave, observed_flux, observed_noise, parm): 
    
    # Loads in the model spectrum
    target_numbers = [4, 4.5, 5, 5.25]
    target_metal = [-2, -1.5, -1, -0.5, -0.25, 0]
    grid_parm = [closest_hundreds(parm[0])[0], closest_hundreds(parm[0])[1], 
                 closest_bounds(parm[1], target_numbers)[0], closest_bounds(parm[1], target_numbers)[1],
                 closest_bounds(parm[2], target_metal)[0], closest_bounds(parm[2], target_metal)[1]]
    model_flux = interpolation(parm, grid_parm, observed_wave, parm[4])

    # Fixes the spectra with null values so it can be normalized and normalizes it
    model_flux = np.nan_to_num(model_flux, nan=2.2250738585072014e-30)
    percentile_995 = np.nanpercentile(model_flux, 99.5)
    model_flux = np.array(model_flux) / percentile_995
    model_flux = [x + parm[3] for x in model_flux]
    model_flux = gaussian_filter1d(model_flux, 1.4)
    
    # Performs the chi square calculations and returns it
    chi_square = np.nansum(((observed_flux - model_flux) / observed_noise) ** 2)
    N, P = len(observed_flux), 5
    reduced_chi_square = chi_square / (N - P)
    return reduced_chi_square

# Loads in the observed spectrum into SPLAT
observed = splat.Spectrum(input_file)

# Gets the observed wavelength
observed_wave = observed.wave.value
observed_wave = observed_wave[int(len(observed_wave)*0.05): len(observed_wave) - int(len(observed_wave)*0.05)]

# Gets the observed noise for the fluxes
observed_noise = [x for x in observed.noise.value]
observed_noise = observed_noise[int(len(observed_noise)*0.05): len(observed_noise) - int(len(observed_noise)*0.05)]

# Gets the observed fluxes
observed_flux = [x for x in observed.flux.value]
observed_flux = observed_flux[int(len(observed_flux)*0.05): len(observed_flux) - int(len(observed_flux)*0.05)]

# Normalizes the observed noise and fluxes
observed_noise = [(x / np.nanmax(observed.flux.value)) for x in observed_noise]
observed_flux = [x / np.percentile(observed_flux, 99.1) for x in observed_flux]
observed_flux = [x if x >= 0 else 2.2250738585072014e-30 for x in observed_flux]
observed_flux = np.nan_to_num(observed_flux, nan=2.2250738585072014e-30)

#### Makes the Parameter Space and Verifies the Walkers are Going in the Correct Direction

In [ ]:
# Estimates temperature based off spectral type estimate
teff_estimate = 2200 - 100*(spt_estimate - 10)
lower_teff = teff_estimate - 500
if lower_teff < 500: 
    lower_teff = 500
upper_teff = teff_estimate + 500
if upper_teff > 5000: 
    upper_teff = 5000

# Defines the parameter space    
def prior(parm):
    a, b, c, d, e = parm
    if lower_teff < a < upper_teff and 4 < b < 6 and -1.5 < c < -0.5 and -0.2 < d < 0.2 and -0.001 < e < 0.001:
        return 0.0
    return -np.inf

# Verifies that the walkers are going in the correct direction
def posterior(parm):
    prior_val = prior(parm)
    if not np.isfinite(prior_val):
        return -np.inf
    return prior_val - 0.5 * chi_square(observed_wave, observed_flux, observed_noise, parm)

#### Set Up the MCMC Code Using the EMCEE Package (MAY TAKE A COUPLE OF MINUTES)

In [ ]:
# Sets the number of parameters, walkers, threads, and steps for the MCMC simulation
ndim, nwalkers = 5, walkers
threads, n_steps = thread, steps

# Makes the random innitial position of the walkers and the MCMC simulation
initial_positions = np.random.uniform(low=[teff_estimate - 100, 5, -1, -0.05, -0.001], high=[teff_estimate + 100, 6, -0.5, 0.05, 0.001], size=(nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, posterior, threads = thread)
 
# Runs the MCMC simulation #
# ------------------------------------------------------------ #
with contextlib.redirect_stdout(io.StringIO()):
    sampler.run_mcmc(initial_positions, n_steps, progress=True)
# ------------------------------------------------------------ #

#### Finds the Best Values and the Errors From the Model

In [ ]:
# Uses EMCEE's discard tool, finds the best parameters, and finds the uncertainties from the simulation
flat_samples = sampler.get_chain(discard=discard, thin=15, flat=True)
best_fit_params = []
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [5, 50, 95])
    best_fit_params.append([mcmc[0], mcmc[1], mcmc[2]])

# Prints these values
fraction_string = f"{'cm'}\u2044{'s^2'}"
chi2_final = chi_square(observed_wave, observed_flux, observed_noise, [best_fit_params[0][1], best_fit_params[1][1], best_fit_params[2][1], best_fit_params[3][1], best_fit_params[4][1]])
print('#--------------------------------------------------------------------------#')
print('                     Best Fit Parameters \u03C72 = ' + str(round((np.median(chi2_final)), 3)))
print('#--------------------------------------------------------------------------#')
print("  Best Fit Parameters: Teff = " + str(round(best_fit_params[0][1], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][1], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][1], 3)))
print('#--------------------------------------------------------------------------#')
print("    Lower Estimate: Teff = " + str(round(best_fit_params[0][0], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][0], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][0], 3)))
print("    Upper Estimate: Teff = " + str(round(best_fit_params[0][2], 1)) + 'K, log(g) = ' + str(round(best_fit_params[1][2], 3)) + str(fraction_string) + ', [M/H] = ' + str(round(best_fit_params[2][2], 3)))
print('#--------------------------------------------------------------------------#')

#### Plots the Comparison of the Spectrum, Walker Space, and a Corner Plot

In [ ]:
# Create the main figure and subplots
fig1, axs = plt.subplots(6, 1, figsize=(10, 15), gridspec_kw={'height_ratios': [4, 1, 1, 1, 1, 1]})

# Plots out the parameter chain for the walkers
# -------------------------------------------------------------- #
# Creates a flattened sample and the labels
samples = sampler.get_chain()
labels = ["$T_{eff}$", "log(g)", "[M/H]", '∆F', '∆λ']

# Plots the data for every parameter
for i in range(ndim):
    # Makes the axis look pretty
    axs[i + 1].plot(samples[:, :, i], "k", alpha=0.4)
    axs[i + 1].set_xlim(0, len(samples))
    axs[i + 1].set_ylabel(labels[i], fontsize = 25, fontname='Times New Roman')
    axs[i + 1].minorticks_on()
    axs[i + 1].grid(True, alpha = 0.5)
    axs[i + 1].tick_params(which='minor', width=1, length=3, labelsize=10)
    axs[i + 1].tick_params(which='major', width = 2, length = 6, labelsize = 10)
    axs[i + 1].tick_params(axis='x', labelsize=9)
    axs[i + 1].tick_params(axis='y', labelsize=9, labelrotation=45)

# Makes the general plot look pretty
axs[5].set_xlabel('Step Number', fontsize = 25, fontname='Times New Roman')

# Saves how the walkers explore into a .csv file
split_arrays = np.split(samples, 5, axis=2)
for i, arr in enumerate(split_arrays):
    flattened_array = arr.reshape((steps, walkers))
    columns = [f'WALKER_{j+1}' for j in range(walkers)]
    df = pd.DataFrame(flattened_array, columns=columns)
    if i == 0: name = 'teff'
    if i == 1: name = 'logg'
    if i == 2: name = 'MH'
    if i == 3: name = 'delta_flux'
    if i == 4: name = 'delta_micron'
    df.to_csv('Output/' + str(output_file) + 'parameter_' + str(name) + '.csv', index=False)
# -------------------------------------------------------------- #

# Plots out the comparison of the observed and model spectra
# -------------------------------------------------------------- #
# Gets the observed data
observed = splat.Spectrum(input_file)
observed.normalize()
observed_wave, observed_flux, observed_noise = observed.wave.value, [x for x in observed.flux.value], [x for x in observed.noise.value]

# Trims the observed data
observed_wave = observed_wave[int(len(observed_wave)*0.05): len(observed_wave) - int(len(observed_wave)*0.05)]
observed_flux = observed_flux[int(len(observed_flux)*0.05): len(observed_flux) - int(len(observed_flux)*0.05)]
observed_noise = observed_noise[int(len(observed_noise)*0.05): len(observed_noise) - int(len(observed_noise)*0.05)]

# Normalize the observed spectrum
observed_noise = [(x / np.nanmax(observed.flux.value)) for x in observed_noise]
observed_flux =  [((x / np.nanpercentile(observed_flux, 99.1))) for x in observed_flux]
observed_flux =  [x if x >= 0 else 2.2250738585072014e-30 for x in observed_flux]

# Gets the best fitted model line fluxes
grid_parm = [closest_hundreds(best_fit_params[0][1])[0], closest_hundreds(best_fit_params[0][1])[1], closest_halves(best_fit_params[1][1])[0], closest_halves(best_fit_params[1][1])[1]]
model_flux = interpolation([best_fit_params[0][1], best_fit_params[1][1], best_fit_params[2][1]], grid_parm, observed_wave, best_fit_params[4][1]) 
model_flux = [(((x) / np.nanpercentile(model_flux, 99.5))) for x in model_flux]

# Plots the data
axs[0].plot(observed_wave, observed_flux, lw=2, c='k', label='Observed')
axs[0].plot(observed_wave, observed_noise, lw=2, c='gray', label='Noise', alpha = 0.3)
model_flux = [x + best_fit_params[3][1] for x in model_flux]
model_flux = gaussian_filter1d(model_flux, 1.4)
if (len(model_flux) - len(observed_wave)) == 0:
    axs[0].plot(observed_wave, model_flux, lw=2, c='lime', label='Best Fit Model')
else: 
    try: 
        axs[0].plot(observed_wave[0:len(model_flux)], model_flux, lw=2, c='lime', label='Best Fit Model')
    except: 
        axs[0].plot(observed_wave, model_flux[0:len(observed_wave)], lw=2, c='lime', label='Best Fit Model')

# Save the DataFrame to a CSV file
df = pd.DataFrame({'WAVELENGTH': observed_wave, 'FLUX': model_flux})
df.to_csv(output_file + '_fitted-spectra.csv', index=False)
        
# Makes the axis look pretty
axs[0].set_xlabel(' WAVELENGTH ($\mu$m)', fontsize = 25, fontname='Times New Roman'), axs[0].set_ylabel('Flux', fontsize = 25, fontname='Times New Roman')
axs[0].minorticks_on()
axs[0].grid(True, alpha = 0.5)
axs[0].tick_params(which='minor', width=1, length=3, labelsize=10)
axs[0].tick_params(which='major', width = 2, length = 6, labelsize = 10)
axs[0].tick_params(axis='x', labelsize=12)
axs[0].tick_params(axis='y', labelsize=12, labelrotation=45)

# Adds a title and legend
legend = axs[0].legend(prop={'family': 'Times New Roman', 'size': 20})
axs[0].set_ylim(-0.1, 1.1)
legend.set_frame_on(False)

# Saves the figure
plt.tight_layout()
plt.savefig('figure1.jpeg')
plt.close('all')
# -------------------------------------------------------------- #

# Plots out the corner plots
# -------------------------------------------------------------- #
# Gets the upper and lower errors
upper_error_t, lower_error_t = round(best_fit_params[0][2] - best_fit_params[0][1], 1), round(best_fit_params[0][1] - best_fit_params[0][0], 1)
upper_error_g, lower_error_g = round(best_fit_params[1][2] - best_fit_params[1][1], 2), round(best_fit_params[1][1] - best_fit_params[1][0], 2)
upper_error_m, lower_error_m = round(best_fit_params[2][2] - best_fit_params[2][1], 2), round(best_fit_params[2][1] - best_fit_params[2][0], 2)
upper_error_f, lower_error_f = round(best_fit_params[3][2] - best_fit_params[3][1], 3), round(best_fit_params[3][1] - best_fit_params[3][0], 3)
upper_error_w, lower_error_w = round(best_fit_params[4][2] - best_fit_params[4][1], 3), round(best_fit_params[4][1] - best_fit_params[4][0], 3)

# Makes the limits for the corner plot
limits = [(best_fit_params[0][0] - (lower_error_t/2), best_fit_params[0][2] + (upper_error_t/2)), 
          (best_fit_params[1][0] - (lower_error_g/2), best_fit_params[1][2] + (upper_error_g/2)), 
          (best_fit_params[2][0] - (lower_error_m/2), best_fit_params[2][2] + (upper_error_m/2)), 
          (best_fit_params[3][0] - (lower_error_f/2), best_fit_params[3][2] + (upper_error_f/2)), 
          (best_fit_params[4][0] - (lower_error_w/2), best_fit_params[4][2] + (upper_error_w/2))]

# Plots the corner plot
figure = corner.corner(flat_samples, labels=labels, truths=[best_fit_params[0][1], best_fit_params[1][1], best_fit_params[2][1], best_fit_params[3][1], best_fit_params[4][1]], 
                       truth_color='lime', title_fmt='.2f', title_kwargs={'fontsize': 25}, plot_contours=True, label_kwargs={'fontsize': 25}, quantiles=[0.05, 0.5, 0.95], use_math_text=True, range=limits)

# Sets the titles for the corner plots
titles = [fr'$Teff = {round(best_fit_params[0][1], 1)}^{{+{upper_error_t}}}_{{-{lower_error_t}}}$' + 'K',
          fr'$log(g) = {round(best_fit_params[1][1], 2)}^{{+{upper_error_g}}}_{{-{lower_error_g}}}$', 
           fr'$[M/H] = -{abs(round(best_fit_params[2][1], 2))}^{{+{upper_error_m}}}_{{-{lower_error_m}}}$', 
           fr'$∆F = {(round(best_fit_params[3][1], 3))}^{{+{upper_error_f}}}_{{-{lower_error_f}}}$', 
           fr'$∆λ = {(round(best_fit_params[4][1], 4))}^{{+{upper_error_w}}}_{{-{lower_error_w}}}$']
list = [0, 6, 12, 18, 24]
for i in range(len(list)):
    ax = figure.axes[list[i]]
    ax.set_title(titles[i], fontsize=20)
    
# Saves the figure
figure = plt.gcf()
figure.set_size_inches(15, 15) 
plt.savefig('figure2.jpeg')
plt.close('all')

# Flattens the sample and saves it
flat_samples = sampler.get_chain(discard=discard, thin=15, flat=True)
df = pd.DataFrame({'FLAT_SAMPLE': flat_samples.tolist()})
df.to_csv('Ouput/' + output_file + '_flat-sample.csv', index=False)
# -------------------------------------------------------------- #

# Combines the 2 pdfs
# -------------------------------------------------------------- #
# Makes function to combine to pdfs side-by-side
def merge_images_side_by_side(image1_path, image2_path, output_path):
    
    # Open the two images
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)

    # Get the sizes of the images
    width1, height1 = image1.size
    width2, height2 = image2.size
    total_width = width1 + width2
    max_height = max(height1, height2)

    # Creates the new merged image
    merged_image = Image.new("RGB", (total_width, max_height))
    merged_image.paste(image1, (0, 0))
    merged_image.paste(image2, (width1, 0))
    merged_image.save(output_path)

# Puts down the image names
image1_path = 'figure1.jpeg'
image2_path = 'figure2.jpeg'
output_path = 'Ouput/' + output_file + '.png'

# Merges the 2 images and removes the temp. state
merge_images_side_by_side(image1_path, image2_path, output_path)
os.remove( 'figure1.jpeg')
os.remove('figure2.jpeg')
# -------------------------------------------------------------- #

# **🎬 END OF SANDCASTL 🎬**